In [8]:
import copy
import json
import hyperopt as hp
import numpy as np
from schema import Optional, Schema
from nni import ClassArgsValidator
from nni.tuner import Tuner
from nni.utils import NodeType, OptimizeMode, extract_scalar_reward, split_index

In [63]:
import collections


In [97]:
tmp = {
    "branch0":{
        "_type":"choice",
        "_value":[
            {
                "_name":"group1",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            },
            {
                "_name":"group2",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            }
        ]
    }
}


In [32]:
tmp = {
    "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
    "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
}

In [7]:
tmp.get()

dict_keys(['branch0', 'branch1'])

In [96]:
def json2space(in_x, name=NodeType.ROOT):
    """
    Change json to search space in hyperopt.

    Parameters
    ----------
    in_x : dict/list/str/int/float
        The part of json.
    name : str
        name could be NodeType.ROOT, NodeType.TYPE, NodeType.VALUE or NodeType.INDEX, NodeType.NAME.
    """
    out_y = copy.deepcopy(in_x)
    if isinstance(in_x, dict):
        if NodeType.TYPE in in_x.keys():
            _type = in_x[NodeType.TYPE]
            name = name + '-' + _type
#             name = _type
            print('nam:{}'.format(name))
            _value = json2space(in_x[NodeType.VALUE], name=name)
            if _type == 'choice':
                out_y = hp.hp.choice(name, _value)
            elif _type == 'randint':
                out_y = hp.hp.randint(name, _value[1] - _value[0])
            else:
                if _type in ['loguniform', 'qloguniform']:
                    _value[:2] = np.log(_value[:2])
                out_y = getattr(hp.hp, _type)(name, *_value)
            print('hhhout_y:{}'.format(out_y))
        else:
            out_y = dict()
            for key in in_x.keys():
                out_y[key] = json2space(in_x[key], name + '[%s]' % str(key))
            print('NodeType.TYPE:{}'.format(NodeType.TYPE))
            print('in_x.keys():{}'.format(in_x.keys()))
            print('strange......, in_x[key]:{}, name:{}, key:{}, '.format(in_x[key], name, key))
        print('out_y:{}'.format(out_y))
    elif isinstance(in_x, list):
        out_y = list()
        print('inner int_x:{}'.format(in_x))
        for i, x_i in enumerate(in_x):
            if isinstance(x_i, dict):
                if NodeType.NAME not in x_i.keys():
                    raise RuntimeError(
                        '\'_name\' key is not found in this nested search space.'
                    )
            if isinstance(x_i, dict):
                out_y.append(json2space(x_i, name))# 
                print('inner:{}, x_i:{}'.format(name, x_i))
            else:
                out_y.append(json2space(x_i, name + '[%d]' % i))# 
                print('inner:{}, x_i:{}'.format(name + '[%d]' % i,x_i))
        
    return out_y

In [99]:
search_space_instance = json2space(tmp)

nam:root[branch0]-choice
inner int_x:[{'_name': 'group1', 'kernel': {'_type': 'choice', '_value': ['linear', 'rbf', 'poly', 'sigmoid']}, 'degree': {'_type': 'choice', '_value': [1, 2, 3, 4]}}, {'_name': 'group2', 'kernel': {'_type': 'choice', '_value': ['linear', 'rbf', 'poly', 'sigmoid']}, 'degree': {'_type': 'choice', '_value': [1, 2, 3, 4]}}]
nam:root[branch0]-choice[kernel]-choice
inner int_x:['linear', 'rbf', 'poly', 'sigmoid']
inner:root[branch0]-choice[kernel]-choice[0], x_i:linear
inner:root[branch0]-choice[kernel]-choice[1], x_i:rbf
inner:root[branch0]-choice[kernel]-choice[2], x_i:poly
inner:root[branch0]-choice[kernel]-choice[3], x_i:sigmoid
hhhout_y:0 switch
1   hyperopt_param
2     Literal{root[branch0]-choice[kernel]-choice}
3     randint
4       Literal{4}
5   Literal{linear}
6   Literal{rbf}
7   Literal{poly}
8   Literal{sigmoid}
out_y:0 switch
1   hyperopt_param
2     Literal{root[branch0]-choice[kernel]-choice}
3     randint
4       Literal{4}
5   Literal{linear}
6   

In [100]:
rstate = np.random.RandomState()
trials = hp.Trials()
domain = hp.Domain(None,
                   search_space_instance,
                   pass_expr_memo_ctrl=None)

rval = hp.FMinIter('tpe',
                    domain,
                    trials,
                    max_evals=-1,
                    rstate=rstate,
                    verbose=0)
print('domain:{}'.format(rval.domain.params))

DuplicateLabel: root[branch0]-choice[degree]-choice

In [72]:
in_x = {
    "branch0":{
        "_type":"choice",
        "_value":[
            {
                "_name":"group1",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            },
            {
                "_name":"group2",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            }
        ]
    }
}

param = {'root[branch0]-choice': 0,\
         'root[branch0]-choice[0][degree]-choice': 3,\
         'root[branch0]-choice[0][kernel]-choice': 3, \
         'root[branch0]-choice[1][degree]-choice': None, \
         'root[branch0]-choice[1][kernel]-choice': None}

In [94]:
def json2parameter(in_x, parameter, name=NodeType.ROOT):
    """
    Change json to parameters.
    把sample出来的结果转成带index的正常的search space的形式
    """
    out_y = copy.deepcopy(in_x)
    if isinstance(in_x, dict):
        if NodeType.TYPE in in_x.keys():
            _type = in_x[NodeType.TYPE]
            name = name + '-' + _type
            if _type == 'choice':
                _index = parameter[name]
                out_y = {
                    NodeType.INDEX:
                    _index,
                    NodeType.VALUE:
                    json2parameter(in_x[NodeType.VALUE][_index],
                                   parameter,
                                   name=name + '[%d]' % _index)
                }
                print('in_x:{},param:{},value:{}'.format(in_x[NodeType.VALUE][_index],parameter,name + '[%d]' % _index))
                print('shenme out_y:{}'.format(out_y))
            else:
                if _type in ['quniform', 'qloguniform']:
                    out_y = np.clip(parameter[name], in_x[NodeType.VALUE][0], in_x[NodeType.VALUE][1])
                elif _type == 'randint':
                    out_y = parameter[name] + in_x[NodeType.VALUE][0]
                else:
                    out_y = parameter[name]
        else:
            out_y = dict()
            for key in in_x.keys():
                print('down key:{}, in_x[key]:{}'.format(key, in_x[key]))
                out_y[key] = json2parameter(in_x[key], parameter,
                                            name + '[%s]' % str(key))
    elif isinstance(in_x, list):
        print('list')
        out_y = list()
        for i, x_i in enumerate(in_x):
            if isinstance(x_i, dict):
                if NodeType.NAME not in x_i.keys():
                    raise RuntimeError(
                        '\'_name\' key is not found in this nested search space.'
                    )
            print('inner?{}'.format(json2parameter(x_i, parameter, name + '[%d]' % i)))
            out_y.append(json2parameter(x_i, parameter, name + '[%d]' % i))
    print('result:{}'.format(out_y))
    return out_y

In [95]:
json2parameter(in_x, param)

down key:branch0, in_x[key]:{'_type': 'choice', '_value': [{'_name': 'group1', 'kernel': {'_type': 'choice', '_value': ['linear', 'rbf', 'poly', 'sigmoid']}, 'degree': {'_type': 'choice', '_value': [1, 2, 3, 4]}}, {'_name': 'group2', 'kernel': {'_type': 'choice', '_value': ['linear', 'rbf', 'poly', 'sigmoid']}, 'degree': {'_type': 'choice', '_value': [1, 2, 3, 4]}}]}
down key:_name, in_x[key]:group1
result:group1
down key:kernel, in_x[key]:{'_type': 'choice', '_value': ['linear', 'rbf', 'poly', 'sigmoid']}
result:sigmoid
in_x:sigmoid,param:{'root[branch0]-choice': 0, 'root[branch0]-choice[0][degree]-choice': 3, 'root[branch0]-choice[0][kernel]-choice': 3, 'root[branch0]-choice[1][degree]-choice': None, 'root[branch0]-choice[1][kernel]-choice': None},value:root[branch0]-choice[0][kernel]-choice[3]
shenme out_y:{'_index': 3, '_value': 'sigmoid'}
result:{'_index': 3, '_value': 'sigmoid'}
down key:degree, in_x[key]:{'_type': 'choice', '_value': [1, 2, 3, 4]}
result:4
in_x:4,param:{'root[br

{'branch0': {'_index': 0,
  '_value': {'_name': 'group1',
   'kernel': {'_index': 3, '_value': 'sigmoid'},
   'degree': {'_index': 3, '_value': 4}}}}

In [101]:
class RandomNumberGenerator():
    def __init__(self):
        np.random.seed(7)

    def integer(high):
        return np.random.randint(0, high-1)

    def uniform(low, high):
        return np.ramdom.uniform(low, high)

    def normal(mu, sigma):
        return np.random.gauss(mu, sigma)

    def categorical(possibility, size=1):
        return np.random.choice(possibility, size).item() if size == 1\
            else np.random.choice(possibility, size)

In [102]:
rng = RandomNumberGenerator()

In [113]:
x = np.random.multinomial(20, , size=(2,))

In [126]:
categorical([1/6.]*6,2)

array([1, 4])

In [129]:
search_space = {
    "branch0":{
        "_type":"choice",
        "_value":[
            {
                "_name":"group1",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            },
            {
                "_name":"group2",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            }
        ]
    },
        "branch1":{
        "_type":"choice",
        "_value":[
            {
                "_name":"group3",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            },
            {
                "_name":"group4",
                "kernel": {"_type":"choice","_value":["linear", "rbf", "poly", "sigmoid"]},
                "degree": {"_type":"choice","_value":[1, 2, 3, 4]}
            }
        ]
    },
}

In [131]:
class ParameterRange():
    def __init__(self, name, algorithm_name, is_categorical, size,
                 categorical_values, low, high, is_log_distributed, is_integer):
        self.name = name
        self.tag = algorithm_name + "|" + name

        self.is_categorical = is_categorical

        self.size = size
        self.categorical_values = categorical_values

        self.low = low
        self.high = high
        self.is_log_distributed = is_log_distributed
        self.is_integer = is_integer

        if is_log_distributed:
            self.high = np.log(high)
            self.low = np.log(low)

    @staticmethod
    def categorical(algorithm_name, name, values):
        return ParameterRange(name, algorithm_name, True,
                              len(values), values, np.nan, np.nan, False, False)

    @staticmethod
    def numerical(algorithm_name, name, low, high,
                  is_log_distributed=False, is_integer=False):
        return ParameterRange(name, algorithm_name, False,
                              -1, None, low, high, is_log_distributed, is_integer)
class SearchSpace():
    def __init__(self, json_string):
        self.algorithms = dict()
        self.pipelines = list()
        for pipeline_json in json_string.values():
            pipeline = list()
            self.pipelines.append(pipeline)
            for algo_kv in pipeline_json["_value"]:
                pipeline.append(algo_kv["_name"])
                algo = list()
                self.algorithms[algo_kv["_name"]] = algo

                for param_kv in list(algo_kv.items())[1:]:
                    param_name = param_kv[0]
                    param_json = param_kv[1]
                    param_type = param_json["_type"]
                    if param_type == "choice":
                        values = list()
                        for val in param_json["_value"]:
                            values.append(val)
                        algo.append(ParameterRange.categorical(
                            algo_kv["_name"], param_name, values))
                    else:
                        values = param_json["_value"]
                        low = values[0]
                        high = values[1]

                        log = (param_type ==
                               "loguniform" or param_type == "qloguniform")
                        integer = (
                            param_type == "quniform" or param_type == "qloguniform")
                        alog.append(ParameterRange.numerical(
                            algo_kv["_name"], param_name, low, high, log, integer))

In [136]:
def suggest(space):
    formatted_param = dict()
    # Parameters : Dictionary<string, AlgorithmParameters>
    # AlgorithmParameters : Dictionary<string, string>
    param = dict()  # Dictionary<string, double>
    # print('len(self.space.pipelines):{}'.format(len(self.space.pipelines)))
    pipeline_index = 0
    chosen_pipeline = space.pipelines[pipeline_index]

    for algo in space.algorithms.items():
        if algo[0] in chosen_pipeline:
            formatted_algo = dict()
            for param_range in algo[1]:
                if param_range.is_categorical:
                    index = 0
                    param[param_range.tag] = index
                    formatted_algo[param_range.name] = param_range.categorical_values[index]
                else:
                    x = 0
                    param[param_range.tag] = x
                    formatted_algo[param_range.name] = param_range.is_integer if str(int(x)) else str(x)

    return formatted_param, param

In [132]:
t = SearchSpace(search_space)

In [137]:
suggest(t)

({},
 {'group1|kernel': 0,
  'group1|degree': 0,
  'group2|kernel': 0,
  'group2|degree': 0})